In [1]:
## Import packages
import os
import csv
import random
import math
import numpy as np
import pandas as pd
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_string(
    'audio_embedding_feature_name', 'audio_embedding',
    'Path to the VGGish checkpoint file.')

## De volgende bestanden komen uit het eerdere mapje 'models'. Nu heb ik ze in yt8m gezet.
flags.DEFINE_string(
    'pca_params', 'models/vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

flags.DEFINE_string(
    'checkpoint', 'models/vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

flags.DEFINE_string(
    'yt_checkpoint', 'models/youtube_model.ckpt',
    'Path to the VGGish checkpoint file.')

In [3]:
# label = "elephants"
# label = "gunshots"
# label = 'bush'
label = 'gun_google'

In [4]:
## Map waarin de wav-files staan
wavfile_path = label
#wavfile_path = 'elephants/wenger_3717'
#wavfile_path = 'elephants/wenger_bush'
## Map waar de nieuwe tf-records terecht komen
tfrecord_path = "tfrecords/added_data/"

In [5]:
## Houd aantal samples bij
nr_9mm = 0
nr_556 = 0
#elephants = 0
#bush = 0
gun_google = 0

## Proportie train/test
pct_train = 0.6

In [6]:
## Voor alle wav-files in een map, gebeurt het volgende:

## Eerst wordt het wav-file ingelezen en de bijbehorende label(s) opgezocht in een csv-bestand
## Als video-id wordt een willekeurig id'tje gegeven (in dit geval allemaal dezelfde)

## Dan parsen we wav-file naar embeddings: 
# (dit gebeurt door vggish_input.wavfile_to_example aan te roepen)
# Stap 1a: lezen van wav-file, input is array met samples die db aanduiden. Ook sample rate (per sec) wordt gelezen
# Stap 1b: Bij 2d array (stereo, ipv mono) bereken gemiddelde, daarna normaliseren (delen door 32.768)
# Stap 2: Bepaal examples in vorm [batch size, num frames, num bands].
    # Hierbij worden voor verschillende batches (omdat alles tegelijk niet in 1x in NN kan),
    # een log mel spectrogram gemaakt (in vorm [num_frames, num_bands])
# Stap 3: Bepaal features: nu wordt de embedding laag gemaakt (PCA-components, discreet maken etc)
    # Hiervoor worden model-parameters opgehaald die eerder zijn opgeslagen
    
## Daarna wordt een sequence example gemaakt (in getSequenceExample) en het als tf-records weggeschreven

In [6]:
## Function that takes examples from wav-file as input and returns a sequence example

def getSequenceExample(examples_batch, labels, video_id=[b'-1LrH01Ei1w']):
    with tf.Graph().as_default(), tf.Session() as sess:

        # Prepare a postprocessor to munge the model embeddings.
        pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)
    
        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)

        # Run inference and postprocessing.
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})
        print(embedding_batch.shape)
        if(embedding_batch.shape!=10):
            print("*****************************************************************")

        postprocessed_batch = pproc.postprocess(embedding_batch)
        #print(postprocessed_batch)


        ## Maak labels en video-id voor in de example
        label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
        videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))

        ## Maak sequence example
        seq_example = tf.train.SequenceExample(
            context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    FLAGS.audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
        
    return(seq_example)

In [8]:
#11,/s/0000,Gunshot SC9mm
#12,/s/0001,Gunshot SC556
#13,/s/0002,Elephant
#14,/s/0003,Bush

def getLabels(file):
    if '9mm' in file:
        labels = [11]
    elif '556' in file or '2000m' in file:
        labels = [12]
    return(labels)

In [9]:
## Lees stuk voor stuk alle wav-files in
## Zoek het bijbehorende label op in een csv-bestand
## Bepaal de embeddings

# Prepare a record writer to store the postprocessed embeddings.
## Trainset
if(label == "gunshots"):
    train_tfrecord = str(tfrecord_path + 'gun_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'gun_test.tfrecord')
elif(label == "elephants"):
    #train_tfrecord = str(tfrecord_path + 'elephant_train.tfrecord')
    #test_tfrecord = str(tfrecord_path + 'elephant_test.tfrecord')
    train_tfrecord = str(tfrecord_path + 'elephant_wenger_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'elephant_wenger_test.tfrecord')
elif(label == "bush"):
    train_tfrecord = str(tfrecord_path + 'bush_wenger_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'bush_wenger_test.tfrecord')
elif(label == "gun_google"):
    train_tfrecord = str(tfrecord_path + 'gun_google_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'gun_google_test.tfrecord')
    
    
    
train_writer = tf.python_io.TFRecordWriter(train_tfrecord)
test_writer = tf.python_io.TFRecordWriter(test_tfrecord)

## Read and rewrite all test files
files = gfile.Glob(str(wavfile_path + "*.wav"))

count = 1
for file in files:
    
    print("file nr: " + str(count))
    print("Filename: " + file)
    
    count += 1
    
    #print("Filename: " + str(file))
   
    ## Find labels belonging to wav-file
    if(label=="gunshots"):
        labels = getLabels(file)
        if(labels[0]==11):
            nr_9mm += 1
        elif(labels[0]==12):
            nr_556 += 1
    elif(label == "elephants"):
        labels = [13]
        elephants += 1
    elif(label == "bush"):
        labels = [14]
        bush += 1
    elif(label == "gun_google"):
        labels = [15]
        gun_google += 1
    
    ## This function reads the wav file and converts the samples into np arrays of [batch size, num frames, num bands]
    #examples_batch = vggish_input.wavfile_to_examples(str(FLAGS.wavfile_path + wav_file))
    examples_batch = vggish_input.wavfile_to_examples(file)
    
    if(examples_batch.shape[0]>10):
        nr_sec = examples_batch.shape[0]
        print(nr_sec)
        start = int(math.floor((nr_sec-10)/2))
        print("start: " + str(start))
        end = int(nr_sec-math.ceil((nr_sec-10)/2))
        print("end: " + str(end))
        examples_batch = examples_batch[start:end, :, :]
        print(examples_batch.shape)
    
    #print("Examples shape: " + str(examples_batch.shape))
   
    seq_example = getSequenceExample(examples_batch, labels)
    
    rand = random.random()
    if rand <= pct_train:
        train_writer.write(seq_example.SerializeToString())
    else:
        test_writer.write(seq_example.SerializeToString())
    
    # Schrijf 556 dubbel weg
    if(labels[0]==12):
        if rand <= pct_train:
            train_writer.write(seq_example.SerializeToString())
        else:
            test_writer.write(seq_example.SerializeToString())
train_writer.close()
test_writer.close()

file nr: 1
Filename: .\gun_google\#OFFICERTITUS S2E1 -  GUNSHOT-8QOCOi3sm-E.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 2
Filename: .\gun_google\'SHOOTER' BY - OTS TSKY FT. GPKCOKEBOY DROOP POP (COKEBOYS)-6Fcj_a9ykc4.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 3
Filename: .\gun_google\'Smith and Wesson M&P - My 2 Cents' by TheGearTester-5uR4ondvGSA.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 4
Filename: .\gun_google\08192014 迫撃砲発射実験-4DK7ypE_cwY.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 5
Filename: .\gun_google\120mm self-propelled mortar system

(10, 128)
*****************************************************************
file nr: 36
Filename: .\gun_google\ARMA 2 - Project Reality Gameplay-3zjEv0432zQ.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 37
Filename: .\gun_google\ArmA 2 TR Fixed Wing Aircraft Sound Replacement. Demo.-6h6NiYJMhXs.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 38
Filename: .\gun_google\ArmA M4A1_M203 sound mods comparison--9XmXoOdb68.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 39
Filename: .\gun_google\ARMA2 Artillery strike The siege of Chernogorsk-4rqB5oTAjPs.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
****************************************

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 71
Filename: .\gun_google\Browning Hi Power Mark III CO2 Pistol - AGR Episode #81-M1xMhk1lCr4.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 72
Filename: .\gun_google\Brutal Doom v20 & Plutonia - Levels 7-9-3OzbRBNJ4HI.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 73
Filename: .\gun_google\Buckshot hidden compartment dining set-1isUtW7FK_E.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 74
Filename: .\gun_google\Buckshot pattern test part 1-8qUMfo_kIjI.wav
INFO:tensorflow:Restoring parameters from models/vggish_mode

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 106
Filename: .\gun_google\CS online 貓小mo 災厄之章四之三 追蹤：最終的絕望 全程篇 2014-03-11 22_13-3DfRg-9tRN8.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 107
Filename: .\gun_google\CSO Korea - NEW MELEE  - Vulcanus 9 _w ZB Bot-7K6UhIewQSo.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 108
Filename: .\gun_google\CYMA Airsoft Tactical Compact M11A1 Tactical Heavyweight SMG review--Zpj-B_JSD8.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 109
Filename: .\gun_google\CZ 200S 4,5mm, 16J, 30m - 5 ran-zu53y8M2xBc.wav
INFO:tensorflow:Resto

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 141
Filename: .\gun_google\Fire For Effect-woVeO6XRBEY.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 142
Filename: .\gun_google\FireSport Sisoev Igor 13.14 Final National Cup 2015 Ufa-5UYsMnVdxpk.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 143
Filename: .\gun_google\Foofding tries - Verdun - Indi WW1 Game-2CslisurMbE.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 144
Filename: .\gun_google\FULL AUTO AK-47 & TOMMY GUN IN LAS VEGAS-4DAlx955cQA.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt


...

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 247
Filename: .\gun_google\Nerf centurion vs buzz bee range master-d61xTv_eJUw.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 248
Filename: .\gun_google\Nerf N-Strike Elite Suction Darts Unboxing, Review & Dart Comparison-Jk2B7Mvh6cE.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 249
Filename: .\gun_google\Nerf Zombie Strike Clear Shot Unboxing-Au5NjQaR_T0.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 250
Filename: .\gun_google\Official CSGO Cup Game #1 PT 1-7dUL3TeelOo.wav
INFO:tensorflow:Restoring parameters from

(10, 128)
*****************************************************************
file nr: 281
Filename: .\gun_google\Retro Rage - Half Life Source Episode 11 - Best Bug Ever!--j1q_NZkYJ8.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 282
Filename: .\gun_google\Review - Taurus PT111 Millennium G2-Fzwa_LIMH2E.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 283
Filename: .\gun_google\Review of The T4 Crosman Air Pistol-jdiphmOtFVU.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 284
Filename: .\gun_google\Rising Storm Gameplay HD-4IUgxXyPnDE.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*******************************************************

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 316
Filename: .\gun_google\Splinter cell essentials Belgrade part 3-1ethZS9uxrg.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 317
Filename: .\gun_google\Splinter Cell Pandora Tomorrow - Walkthrough - mission 5 - Kundang, Indonesia - Hard - part 2-1f4mp65fpyE.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 318
Filename: .\gun_google\Springfield XD .45 - Initial Shot Groups At 15 Yards-7qeEs3ahSm4.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 319
Filename: .\gun_google\SR556 With The Slide Fire Solutions Stock-3ruqib

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 350
Filename: .\gun_google\Top 10 Scariest Moments in Games-284W5cxWL2Q.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 351
Filename: .\gun_google\Toy Wars Invasion Gameplay (PC HD)-8rnITQpV_Js.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 352
Filename: .\gun_google\Transformers 3 shoot in Chicago - base jumping_gunfire-1ywJflmdH1c.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 353
Filename: .\gun_google\Troy Defense 5.56 Carbine-4kJIAn9qBlU.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10,

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 384
Filename: .\gun_google\[29th ID] Charlie Fight Night - Artillery Defence - Arma 3-4yvS6NRKn_o.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 385
Filename: .\gun_google\[Company of Heroes 2] Sector Artillery Pounding-6xRqi2JC36k.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 386
Filename: .\gun_google\[Maple Story] Maple SMKCP episode 1-8Egof932hUw.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 387
Filename: .\gun_google\_-1Wz31eLYuk8.wav
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128

In [10]:
print("nr 9mm: " + str(nr_9mm))
print("nr 556: " + str(nr_556))

nr 9mm: 0
nr 556: 0


In [16]:
print("nr elephants: " + str(elephants))

nr elephants: 3717


In [20]:
print("nr bush: " + str(bush))

nr bush: 1540


In [10]:
print("nr gun google: " + str(gun_google))

nr gun google: 414
